In [8]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import time
from scipy import linalg
from scipy import optimize
import sympy as sm

# local module for linear algebra
#%load_ext autoreload
#%autoreload 2
import numecon_linalg


In [9]:
p_t = sm.symbols("P_t")
d_t = sm.symbols("d_t")
b_t = sm.symbols("b_t")
a_t = sm.symbols("a_t")
v = sm.symbols("v")
vh = sm.symbols("v**H")
vl = sm.symbols("v**L")
theta_t = sm.symbols("theta_t")
my_t = sm.symbols("my_t")
pi = sm.symbols("pi")

In [16]:
dealer_profit = theta_t*pi*(a_t-vh)+1/2*(1-pi)*(a_t-my_t)

In [17]:
print(sm.latex(dealer_profit))

\pi \theta_{t} \left(a_{t} - v**H\right) + \left(a_{t} - my_{t}\right) \left(- 0.5 \pi + 0.5\right)


\pi \theta_{t} \left(a_{t} - v**H\right) + \left(a_{t} - my_{t}\right) \left(- 0.5 \pi + 0.5\right)
